# project2의 skeleton 코드입니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

def open_pickle(path: str):
    with open(path, 'rb') as f:
        target = pickle.load(f)
    return target

## 1. train, valid 데이터 셋 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd "/content/drive/MyDrive/정보보호와시스템보안"

/content/drive/MyDrive/정보보호와시스템보안


In [5]:
train_path = r"./train.csv"
valid_path = r"./test.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [6]:
train_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,21.122.91.166,13850,30.239.23.122,53,TCP,......S.,03:52:00,3119967,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,21.122.91.166,43842,30.239.23.122,53,TCP,......S.,03:37:20,2991830,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,21.122.91.165,25918,30.239.23.122,53,TCP,......S.,03:10:28,3004658,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
3,173.123.45.117,4445,30.239.23.122,53,TCP,...A...F,00:42:24,4402645,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,21.122.91.165,28467,30.239.23.122,53,TCP,......S.,02:45:28,3001784,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


In [7]:
valid_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,21.122.91.166,23801,30.239.23.122,53,TCP,......S.,03:33:43,2987074,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,21.122.91.166,28413,30.239.23.122,53,TCP,......S.,03:32:49,2977944,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,21.122.91.166,49550,30.239.23.122,53,TCP,......S.,04:23:46,2995375,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
3,21.122.91.166,38373,30.239.23.122,53,TCP,......S.,04:28:08,3003635,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,21.122.91.165,8805,30.239.23.122,53,TCP,......S.,03:46:39,3006388,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


## 2. feature 추가하기

도메인 지식을 활용하여 feature를 자유롭게 추가하시면 됩니다.

In [8]:
label_groups = {label: group for label, group in train_df.groupby("Label")}
normal_data = label_groups['Benign_IP']
attack_data = label_groups['Attack_IP']

In [9]:
summary_normal = normal_data.describe()
summary_attack = attack_data.describe()

sum_summary = summary_normal + summary_attack
diff_summary = summary_normal - summary_attack
print(diff_summary / sum_summary)

       Src Port  Dst Port  Flow Duration  Total Fwd Packet  Total Bwd packets  \
count  0.168836  0.168836       0.168836          0.168836           0.168836   
mean  -0.033986 -0.367769       0.531818         -0.383915           0.822537   
std    0.146802 -0.204765       0.106062          0.558484           0.697098   
min         NaN       NaN            NaN          0.000000                NaN   
25%   -0.105763 -0.929351       1.000000          0.000000                NaN   
50%   -0.016845 -0.441877       0.990078          0.200000                NaN   
75%    0.027420 -0.468769       0.999122         -0.921053           0.000000   
max    0.000053 -0.000298       0.000003          0.895188           0.711297   

       Total Length of Fwd Packet  Total Length of Bwd Packet  \
count                    0.168836                    0.168836   
mean                     0.611501                    0.949185   
std                      0.809441                    0.886605   
min       

# **데이터 전처리**
1. 특정 칼럼을 뽑아쓰는 방식은 힘들어 보인다. -> 각 열 데이터에 따라 크게 변화가 있지 않아보였다.
2. 그중에서 크게 의미없다고 생각되는 특정 열만 제외하자.
3. IP, flag 등과 같은 데이터는 가능한 숫자형태로 바꿔서 넣어주자

In [10]:
import ipaddress
from sklearn.preprocessing import LabelEncoder

In [11]:
#IP subnet 기반의 flag 만들기
def ip_map(ip):
  first_octet = int(ip.split('.')[0])
  if(1 <= first_octet <= 126):
    return 1
  elif 127 <= first_octet <= 191:
    return 2
  elif 192 <= first_octet <= 224:
    return 3
  elif 225 <= first_octet <= 240:
    return 4
  else:
    return 5

In [12]:
#IP -> 정수형 변경
def convert_ip(ip):
  return int(ipaddress.IPv4Address(ip))

In [13]:
#df (pd.DataFrame): 데이터프레임
#column_name (str): 인코딩할 컬럼 이름
#new_column_name (str): 생성할 컬럼 이름

def encode_column(df, column_name, new_column_name):
    encoder = LabelEncoder()
    df[new_column_name] = encoder.fit_transform(df[column_name])
    # df.drop(columns=[column_name], inplace=True)
    return df

In [14]:
# df (pd.DataFrame): 데이터프레임
# ip_column (str): IP 주소 컬럼 이름
# prefix (str): 새 컬럼 이름의 접두사 ('Src' 또는 'Dst')

def preprocess_ip(df, ip_column, prefix):
    df[f'{prefix}_IP_map'] = df[ip_column].apply(ip_map)
    df[f'{prefix}_Int_IP'] = df[ip_column].apply(convert_ip)
    # df.drop(columns=[ip_column], inplace=True)
    return df

In [15]:
def preprocess_dataset(df):
    # IP 전처리
    df = preprocess_ip(df, 'Src IP', 'Src')
    df = preprocess_ip(df, 'Dst IP', 'Dst')

    # Flags 인코딩
    df = encode_column(df, 'Flags', 'Encoded_Flags')

    # Protocol 인코딩
    df = encode_column(df, 'Protocol', 'Encoded_Protocol')

    return df

In [16]:
train_y = [0 if i == "Benign_IP" else 1 for i in train_df['Label']]
valid_y = [0 if i == "Benign_IP" else 1 for i in valid_df['Label']]

### 스케일러 적용

In [17]:
except_col = ['Timestamp', 'Label', 'Fwd Packet Length Std', 'Active Std', 'Src IP', 'Protocol', 'Bwd Packet Length Std', 'Flags', 'Bwd IAT Std', 'Dst IP']

In [18]:
train_expt = preprocess_dataset(train_df)
train_expt = train_df.drop(except_col, axis = 1)
train_list = train_expt.values.tolist()

valid_expt = preprocess_dataset(valid_df)
valid_expt = valid_df.drop(except_col, axis = 1)
valid_list = valid_expt.values.tolist()

In [19]:
print(len(train_expt.keys()))

50


In [20]:
# standardScaler
from sklearn.preprocessing import StandardScaler

normalizer = StandardScaler()
train_X = []
valid_X = []

normalizer.fit(train_list)

train_X = normalizer.transform(train_list)
valid_X = normalizer.transform(valid_list)

In [21]:
#학습을 위한 데이터 타입 변환

train_X = train_X.astype(np.float32)
valid_X = valid_X.astype(np.float32)
train_y = np.array(train_y, dtype=np.float32)
valid_y = np.array(valid_y, dtype=np.float32)

977270, 325560 이 나와야 함.

In [22]:
len(train_y), len(valid_y)

(977270, 325963)

# 3. 모델 학습

# **RandomForest 기반**

In [23]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_estimators = 100, random_state = 108) # randomstate는 108로 고정 0.97
# rfc.fit(train_X, train_y)
# y_pred = rfc.predict(valid_X)

플로우 기반 이상탐지 성능평가

In [24]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# # 성능 평가를 위해 예측 값을 반올림
# round_valid_y = valid_y.astype("int")
# round_y_pred = np.round(y_pred)
# print("confusion_matrix\n", confusion_matrix(round_valid_y, round_y_pred))
# print(f"accuracy\t: {accuracy_score(round_valid_y, round_y_pred): .5f}")
# print(f"precision\t: {precision_score(round_valid_y, round_y_pred): .5f}")
# print(f"recall\t\t: {recall_score(round_valid_y, round_y_pred): .5f}")
# print(f"F-1\t\t: {f1_score(round_valid_y, round_y_pred): .5f}")

# **Lightgbm**

In [25]:
from lightgbm import LGBMClassifier
#{'n_estimators': 958, 'learning_rate': 0.05367520221347052, 'max_depth': 5, 'num_leaves': 280}. Best is trial 0 with value: 0.999339599459393. 091
# {'n_estimators': 602, 'learning_rate': 0.09769982100415123, 'max_depth': 5, 'num_leaves': 201}. Best is trial 6 with value: 0.9993764590244502.
# {'n_estimators': 348, 'learning_rate': 0.09359221453769044, 'max_depth': 9, 'num_leaves': 890}. Best is trial 6 with value: 0.9993764590244502.
# {'n_estimators': 789, 'learning_rate': 0.08614694420893249, 'max_depth': 9, 'num_leaves': 970}. Best is trial 5 with value: 0.9993703157636074.
# {'n_estimators': 958, 'learning_rate': 0.05367520221347052, 'max_depth': 5, 'num_leaves': 280}. Best is trial 0 with value: 0.999339599459393.
# {'n_estimators': 568, 'learning_rate': 0.06635960798264401, 'max_depth': 8, 'num_leaves': 757}. Best is trial 0 with value: 0.999339599459393.
# {'n_estimators': 321, 'learning_rate': 0.013961855681780553, 'max_depth': 8, 'num_leaves': 635}. Best is trial 0 with value: 0.999339599459393.
# {'n_estimators': 610, 'learning_rate': 0.05819869111299436, 'max_depth': 10, 'num_leaves': 952}. Best is trial 0 with value: 0.999339599459393.
# 0.9993764590244502 and parameters: {'n_estimators': 348, 'learning_rate': 0.09359221453769044, 'max_depth': 9, 'num_leaves': 890}. Best is trial 6 with value: 0.9993764590244502. 995
lgbmc = LGBMClassifier(random_state = 108, n_estimators = 348, learning_rate = 0.09359221453769044, max_depth = 9, num_leaves = 890, objective = 'binary', metric = 'binary_logloss')
lgbmc.fit(train_X, train_y)
y_pred = lgbmc.predict(valid_X)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 406136, number of negative: 571134
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.232584 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10977
[LightGBM] [Info] Number of data points in the train set: 977270, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.415582 -> initscore=-0.340936
[LightGBM] [Info] Start training from score -0.340936
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 성능 평가를 위해 예측 값을 반올림
round_valid_y = valid_y.astype("int")
round_y_pred = np.round(y_pred)
print("confusion_matrix\n", confusion_matrix(round_valid_y, round_y_pred))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_y_pred): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_y_pred): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_y_pred): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_y_pred): .5f}")

confusion_matrix
 [[190374    117]
 [   105 135367]]
accuracy	:  0.99932
precision	:  0.99914
recall		:  0.99922
F-1		:  0.99918


# 4. IP voting

실제 성능평가는 IP기준으로 해야하기 때문에 flow 기반 예측결과를 이용하여 IP votiong을 진행합니다.

In [27]:
# 정상 외부 IP set
outer_benignIP_set = open_pickle("./outer_benign_IP_train.pkl")
# 악성 외부 IP set
outer_malIP_set = open_pickle("./outer_mal_IP_train.pkl")

In [28]:
#외부 IP의 악성, 정상 개수 확인
len(outer_benignIP_set), len(outer_malIP_set)

(590, 457)

In [29]:
# Problem #3을 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [30]:
def group_external_ip(df, external_IP):
    df.rename(columns={'Src IP':'src_ip','Dst IP':'dst_ip'}, inplace=True)
    external_IP_group = {}
    for row in df.itertuples():
          src_ip = row.src_ip
          dst_ip = row.dst_ip

          if src_ip in external_IP:
            if src_ip in external_IP_group:
              external_IP_group[src_ip].append(row)
            else:
              external_IP_group[src_ip] = [row]
          else:
            if dst_ip in external_IP_group:
              external_IP_group[dst_ip].append(row)
            else:
              external_IP_group[dst_ip] = [row]

    return external_IP_group

In [31]:
def predict_external_ip(external_IP_group, y_pred, proportion = 0.65):
    predictIP_dict = {}

    for key, values in external_IP_group.items():
        predictIP_dict[key] = 0
        cnt = 0

        for value in values:
            if y_pred[value.Index] == 1:
                cnt += 1
                if cnt / float(len(values)) >= proportion:
                    predictIP_dict[key] = 1
                    break
    return predictIP_dict

In [32]:
predictIP_dict = dict()
external_IP =list(labelIP_dict.keys())
external_IP_group = group_external_ip(valid_df, external_IP)
predictIP_dict = predict_external_ip(external_IP_group, y_pred, proportion = 0.65)

In [33]:
pred_IP = []
label_IP = []

for k in predictIP_dict.keys():
    pred_IP.append(1 if predictIP_dict[k] == 1 else 0)
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[587   3]
 [  1 456]]
accuracy	:  0.99618
precision	:  0.99346
recall		:  0.99781
F-1		:  0.99563


# 5. 제출 csv 생성

In [34]:
%cd "/content/drive/MyDrive/정보보호와시스템보안"

/content/drive/MyDrive/정보보호와시스템보안


In [35]:
test_path = r"/content/drive/MyDrive/정보보호와시스템보안/test.csv"
test_df = pd.read_csv(test_path)

In [36]:
test_expt = preprocess_dataset(test_df)
test_expt = test_df.drop(except_col, axis = 1)
test_list = test_expt.values.tolist()

In [37]:
test_X = normalizer.transform(test_list)

In [38]:
test_X = test_X.astype(np.float32)
print(len(test_X))

325963


In [39]:
# y_pred_test = rfc.predict(test_X)
y_pred_test = lgbmc.predict(test_X)

In [40]:
test_y = [0 if i == "Benign_IP" else 1 for i in test_df['Label']]
test_y = np.array(test_y, dtype=np.float32)

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 성능 평가를 위해 예측 값을 반올림
round_test_y = test_y.astype("int")
round_y_pred = np.round(y_pred_test)
print("confusion_matrix\n", confusion_matrix(round_test_y, round_y_pred))
print(f"accuracy\t: {accuracy_score(round_test_y, round_y_pred): .5f}")
print(f"precision\t: {precision_score(round_test_y, round_y_pred): .5f}")
print(f"recall\t\t: {recall_score(round_test_y, round_y_pred): .5f}")
print(f"F-1\t\t: {f1_score(round_test_y, round_y_pred): .5f}")

confusion_matrix
 [[190374    117]
 [   105 135367]]
accuracy	:  0.99932
precision	:  0.99914
recall		:  0.99922
F-1		:  0.99918


In [42]:
outer_test_all = open_pickle("./outer_IP_test_all.pkl")

In [43]:
len(outer_test_all)

1047

In [44]:
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

print(len(labelIP_dict))

1047


In [49]:
predictIP_dict_test = dict()
external_IP_test =list(labelIP_dict.keys())
external_IP_group = group_external_ip(test_df, external_IP_test)
predictIP_dict_test = predict_external_ip(external_IP_group, y_pred_test, proportion = 0.65)

pred_IP = []
label_IP = []

print(len(predictIP_dict_test))

1047


In [50]:
for k in predictIP_dict_test.keys():
    pred_IP.append(1 if predictIP_dict_test[k] == 1 else 0)
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[587   3]
 [  1 456]]
accuracy	:  0.99618
precision	:  0.99346
recall		:  0.99781
F-1		:  0.99563


In [51]:
len(predictIP_dict_test)

1047

In [52]:
import csv
csv_file = './test_result.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    header = ["IP", "predict"]
    writer.writerow(header)

    for key, value in predictIP_dict_test.items():
        writer.writerow([key, value])